In [18]:
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html

Looking in links: https://download.pytorch.org/whl/lts/1.8/torch_lts.html


In [20]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [21]:
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

ln: failed to create symbolic link '/mydrive/My Drive': File exists
 1707008
 1707026
 1707026.pdf
 1707026.zip
 1707040
 1707059
 1e93121c-7ed4-4f3b-ac48-93cd4d673968.jpg
'2.1 math'
 3.1
 3.2
'Azhar Sir CT'
'Bandarban 2021'
 Classroom
'Colab Notebooks'
'Compiler Project'
'CSE 2k17 Signatures'
'CSE 3212'
'CSE 3212 Compiler Design Lab Project Reports 15 June.zip'
'CSE 3212 Project Report'
 dataset
'Family Get-together'
'Fish n Chips'
'Getting started.pdf'
 HR
'Hum Assignments'
 IMG_20210101_101719.jpg
 IMG_20210101_170308.jpg
 Maruf
'MBA Thesis Works'
'My Drive'
"Nazia Ma'am Assignment"
 NID.pdf
 NormalReplyBot
'Paper_1 (1).docx'
 Paper_1.docx
 Paper_2.docx
'Rokib Sir (17-05).zip'
'Rokib Sir Assignment'
'saaint martin tour'
"Sabab Vaiya's Engagement"
 Screenrecorder-2021-08-03-00-40-24-682.mp4
'season 3'
 test
'ToC CT'
 WP
 yolov4
'পেদা টিং টিং'


In [22]:
!pip3 install pip==20.1.1

In [23]:
!pip3 install git+https://github.com/PrithivirajDamodaran/Gramformer.git

  Cloning https://github.com/PrithivirajDamodaran/Gramformer.git to /tmp/pip-req-build-bmfj8pru
  Running command git clone -q https://github.com/PrithivirajDamodaran/Gramformer.git /tmp/pip-req-build-bmfj8pru
  Created wheel for gramformer: filename=gramformer-1.0-py3-none-any.whl size=4501 sha256=d636462c4fc05e1245c8ac3e4317c60deed6a264648ad4ccdae0a4d5580cbabc
  Stored in directory: /tmp/pip-ephem-wheel-cache-75wu0ec2/wheels/c1/f6/04/a7ceda8a8c1d494a3523957d61eb6dfe428568b1af0b9d508c
Successfully built gramformer


In [24]:
from gramformer import Gramformer

In [25]:
from gramformer import Gramformer
import torch

def set_seed(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(1212)


gf = Gramformer(models = 1, use_gpu=False) # 1=corrector, 2=detector

influent_sentences = [
    "He are moving here.",
    "I am doing fine. How is you?",
    "How is they?",
    "Matt like fish",
    "the collection of letters was original used by the ancient Romans",
    "We enjoys horror movies",
    "Anna and Mike is going skiing",
    "I walk to the store and I bought milk",
    " We all eat the fish and then made dessert",
    "I will eat fish for dinner and drink milk",
    "what be the reason for everyone leave the company",
]   

for influent_sentence in influent_sentences:
    corrected_sentences = gf.correct(influent_sentence, max_candidates=1)
    print("[Input] ", influent_sentence)
    for corrected_sentence in corrected_sentences:
      print("[Correction] ",corrected_sentence)
    print("-" *100)

[Gramformer] Grammar error correct/highlight model loaded..
[Input]  He are moving here.
[Correction]  ('He is moving here.', -31.02849578857422)
----------------------------------------------------------------------------------------------------
[Input]  I am doing fine. How is you?
[Correction]  ('I am doing fine, how are you?', -37.671016693115234)
----------------------------------------------------------------------------------------------------
[Input]  How is they?
[Correction]  ('How are they?', -24.648284912109375)
----------------------------------------------------------------------------------------------------
[Input]  Matt like fish
[Correction]  ('Matt likes fish.', -33.768829345703125)
----------------------------------------------------------------------------------------------------
[Input]  the collection of letters was original used by the ancient Romans
[Correction]  ('the collection of letters was original used by the ancient Romans', -63.627716064453125)
--------

In [26]:
import re


def clean_text(txt):
    txt = txt.lower()
    txt = re.sub(r"i'm", "i am", txt)
    txt = re.sub(r"it's", "it is", txt)
    txt = re.sub(r"he's", "he is", txt)
    txt = re.sub(r"she's", "she is", txt)
    txt = re.sub(r"that's", "that is", txt)
    txt = re.sub(r"what's", "what is", txt)
    txt = re.sub(r"where's", "where is", txt)
    txt = re.sub(r"\'ll", " will", txt)
    txt = re.sub(r"\'ve", " have", txt)
    txt = re.sub(r"\'re", " are", txt)
    txt = re.sub(r"\'d", " would", txt)
    txt = re.sub(r"won't", "will not", txt)
    txt = re.sub(r"can't", "can not", txt)

    return txt


def create_training_data():
    data_path = r'/mydrive/Maruf/Final_Everything/Arcadia/human_text.txt'
    data_path2 = r'/mydrive/Maruf/Final_Everything/Arcadia/robot_text.txt'
    # Defining lines as a list of each line
    with open(data_path, "r", encoding='utf-8') as f:
        lines = f.read().split('\n')
    with open(data_path2, "r", encoding='utf-8') as f:
        lines2 = f.read().split('\n')
    lines = [re.sub(r"\[\w+\]", 'hi', line) for line in lines]
    lines = [" ".join(re.findall(r"\w+", line)) for line in lines]
    lines2 = [re.sub(r"\[\w+\]", '', line) for line in lines2]
    lines2 = [" ".join(re.findall(r"\w+", line)) for line in lines2]

    encoder_input_data = []
    decoder_input_data = []
    decoder_output_data = []


    for i in range(len(lines)):
        encoder_input_data.append(clean_text(lines[i]))
        decoder_input_data.append('<sos> ' + clean_text(lines2[i]))
        decoder_output_data.append(clean_text(lines2[i]) + ' <eos>')
    return encoder_input_data, decoder_input_data, decoder_output_data


create_training_data()


VOCAB_SIZE = 5000
MAXLEN = 40
EPOCHS = 150
BATCH_SIZE = 1024
VERBOSE = 1
SAVE_AT = 50
LEARNING_RATE = 0.01
LOSS = 'sparse_categorical_crossentropy'
a,b,c=create_training_data()
c[:5]

['hi there how are you <eos>',
 'here is afternoon <eos>',
 'my name is rdany but you can call me dany the r means robot i hope we can be virtual friends <eos>',
 'i have many but not enough to fully understand humans beings <eos>',
 'i ve talked with 143 users counting 7294 lines of text <eos>']

In [27]:
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Bidirectional, Embedding
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import Input
from tensorflow.keras.models import load_model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Lambda


def seq2seq():
    encoder_inputs= Input(shape=(40,))
    encoder_embedding = Embedding(VOCAB_SIZE, 100, input_length=MAXLEN)

    decoder_embedding = Embedding(VOCAB_SIZE, 100, input_length=MAXLEN)
    encoder_embeddings = encoder_embedding(encoder_inputs)
    encoder_lstm=LSTM(256, return_state=True, kernel_regularizer=l2(0.0000001), activity_regularizer=l2(0.0000001))
    LSTM_outputs, state_h, state_c = encoder_lstm(encoder_embeddings)

    encoder_states = [state_h, state_c]

    decoder_inputs = Input(shape=(40,), name='decoder_inputs')
    decoder_lstm = LSTM(256, return_sequences=True, return_state=True, name='decoder_lstm', kernel_regularizer=l2(0.0000001), activity_regularizer=l2(0.0000001))
    decoder_embeddings = decoder_embedding(decoder_inputs)
    decoder_outputs, _, _ = decoder_lstm(decoder_embeddings,
                                         initial_state=encoder_states)


    decoder_dense = Dense(5000, activation='softmax', name='decoder_dense')



    decoder_outputs = decoder_dense(decoder_outputs)
    print(decoder_outputs)
    seq2seq = Model([encoder_inputs, decoder_inputs], decoder_outputs, name='model_encoder_training')

    return seq2seq

In [28]:
import pickle
import re
import tensorflow as tf



class Tokenizer:
    def __init__(self, vocab_size=10000, maxlen=40, padding='post'):

        self.tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=VOCAB_SIZE, oov_token="<unk>",
                                                               filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ', split=' ')
        self.maxlen = maxlen
        self.padding = padding

    def tokenize_and_pad_training_data(self, encoder_input_data, decoder_input_data, decoder_output_data):
        text_corpus = encoder_input_data + decoder_input_data + decoder_output_data
        self.tokenizer.fit_on_texts(text_corpus)

        self.tokenizer.word_index['<pad>'] = 0
        self.tokenizer.index_word[0] = '<pad>'

        encoder_input_data_tokenized = self.tokenizer.texts_to_sequences(encoder_input_data)
        decoder_input_data_tokenized = self.tokenizer.texts_to_sequences(decoder_input_data)
        decoder_output_data_tokenized = self.tokenizer.texts_to_sequences(decoder_output_data)

        encoder_input_data_padded = tf.keras.preprocessing.sequence.pad_sequences(encoder_input_data_tokenized,
                                                                                  padding=self.padding,
                                                                                  maxlen=self.maxlen)
        decoder_input_data_padded = tf.keras.preprocessing.sequence.pad_sequences(decoder_input_data_tokenized,
                                                                                  padding=self.padding,
                                                                                  maxlen=self.maxlen)
        decoder_output_data_padded = tf.keras.preprocessing.sequence.pad_sequences(decoder_output_data_tokenized,
                                                                                   padding=self.padding,
                                                                                   maxlen=self.maxlen)

        return encoder_input_data_padded, decoder_input_data_padded, decoder_output_data_padded

    def decode_sequence(self, encoded_text):
        lst = []
        for i in encoded_text:
            lst.append(self.tokenizer.index_word[i])
        return ' '.join(lst)

    def tokenize_sequence(self, sequence):
        tokenized_sequence = []
        sequence = sequence.lower()
        sequence = sequence.strip()
        sequence = re.sub(r'[^\w\s]', '', sequence)
        for i in sequence.split(' '):
            try:
                tokenized_sequence.append(self.tokenizer.word_index[i])
            except:
                tokenized_sequence.append(self.tokenizer.word_index['well'])
        if len(tokenized_sequence) > 40:
            tokenized_sequence = tokenized_sequence[:40]
        elif len(tokenized_sequence) == 40:
            tokenized_sequence = tokenized_sequence
        else:
            length = len(tokenized_sequence)
        for i in range(40 - length):
            tokenized_sequence.append(0)
        return tokenized_sequence

    def save_tokenizer(self, name):
        with open(f'{name}.pickle', 'wb') as handle:
            pickle.dump(self.tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

    def load_tokenizer(self, path):
        with open(path, 'rb') as handle:
            tokenizer = pickle.load(handle)
            self.tokenizer = tokenizer
        return tokenizer

In [29]:
from math import log
import numpy as np
from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import LSTM, Embedding

# +++++++++++++++++++++++++++++++++ seq2seq model to refere layers with their names ++++++++++++++++++++++++++++++++
encoder_inputs = Input(shape=(40,))
encoder_embedding = Embedding(VOCAB_SIZE, 100, input_length=MAXLEN)

decoder_embedding = Embedding(VOCAB_SIZE, 100, input_length=MAXLEN)
encoder_embeddings = encoder_embedding(encoder_inputs)
encoder_lstm = LSTM(256, return_state=True, kernel_regularizer=l2(0.0000001), activity_regularizer=l2(0.0000001))
LSTM_outputs, state_h, state_c = encoder_lstm(encoder_embeddings)

encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(40,), name='decoder_inputs')
decoder_lstm = LSTM(256, return_sequences=True, return_state=True, name='decoder_lstm',
                    kernel_regularizer=l2(0.0000001), activity_regularizer=l2(0.0000001))
decoder_embeddings = decoder_embedding(decoder_inputs)
decoder_outputs, _, _ = decoder_lstm(decoder_embeddings, initial_state=encoder_states)

decoder_dense = Dense(5000, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_outputs)

Seq2SeqModel = Model([encoder_inputs, decoder_inputs], decoder_outputs, name='model_encoder_training')

# +++++++++++++++++++++++++++++++++ model for predictions +++++++++++++++++++++++++++++++++
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_inputs = Input(shape=(1,))
embedded = decoder_embedding(decoder_inputs)
decoder_outputs, state_h, state_c = decoder_lstm(embedded, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)


# +++++++++++++++++++++++++++++++++ Predict Class +++++++++++++++++++++++++++++++++
class Predict():
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer

    def create_response(self, question):
        question = np.expand_dims(self.tokenizer.tokenize_sequence(clean_text(question)), axis=0)
        result = self.predict_sentence(question)
        return result

    def predict_sentence(self, input_seq):
        with tf.device('/cpu:0'):
            states_value = encoder_model.predict(input_seq)

            target_seq = np.zeros((1, 1))
            target_seq[0, 0] = self.tokenizer.tokenizer.word_index['<sos>']
            output_sentence = []

            for _ in range(MAXLEN):
                output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
                idx = np.argmax(output_tokens)

                if self.tokenizer.tokenizer.index_word[idx] == '<eos>':
                    break

                output_sentence.append(idx)
                target_seq[0, 0] = idx
                states_value = [h, c]

        return self.tokenizer.decode_sequence(output_sentence)

In [38]:
tokenizer = Tokenizer()

# loading tokenizer
tokenizer.load_tokenizer('/mydrive/Maruf/Final_Everything/Arcadia/tokenizer-vocab_size-5000.pickle')

# loading pretrained weight
Seq2SeqModel.load_weights('/mydrive/Maruf/Final_Everything/Arcadia/seq2seq-weights-800-epochs-0.01-learning_rate.h5')

predict = Predict(Seq2SeqModel, tokenizer)

def chatwithbot(text):
    return (predict.create_response(text))


print("Hello")

while True:
  text=input()
  corrected_text=gf.correct(text)
  
  #print(type(corrected_text[0][0]))
  #)
  if ( text == "name?" or text == "Name?" or text == "Your name?" or text == "your name?" or text == "Tell me your name" or text =="tell me your name"):
    reply = "My name is rdany but you can call me dany."
    print(reply)
  elif ( text == "no one loves me" or text == "I feel I don't know"):
    reply = "There's always someone for everyone. Don't do something stupid."
    print(reply)
  elif ( text == "depressed" or text == "I am depressed" or text == "I want to kill myself"):
    reply = "don't be depressed. Talk to someone professional or someone you love. There's always someone for everyone. Don't do something stupid."
    print(reply)
  elif ( text == corrected_text[0][0]):
     reply=chatwithbot(corrected_text[0][0] )
     print(reply)
  else:
      print("Did you mean: ")
      print(corrected_text[0][0])
      yn=input()
      if(yn=="yes" or yn=="Yes"):
          reply=chatwithbot(corrected_text[0][0])
          print(reply)
      
  

Hello
tell me your name
My name is rdany but you can call me dany.
hi dany
Did you mean: 
hi dany santa jo afos.
no
hello dany.
hahaha
I am depressed
don't be depressed. Talk to someone professional or someone you love. There's always someone for everyone. Don't do something stupid.
okay
Did you mean: 
okay?
no
okay
Did you mean: 
okay.
yes
and how it work
don't know.
oh so is a lot of the time hahaha is good or fun do you like it
okay it was good talking to you.
Did you mean: 
okay, it was good to talk to you.
yes
python


KeyboardInterrupt: ignored

In [ ]:
print(type(text))

In [ ]:
sentences = [
    'I like for walks', 
    'World is flat', 
    'Red a color', 
    'I wish my Computer was run faster.'
]

In [ ]:
for sentence in sentences:
    res = gf.correct(sentence)
    print(res[0])